In [85]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config
from pyeoskit._hello import _eosapi

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='helloworld11'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)


def set_code(account_name, code):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash == r['code_hash']:
        return

    setcode = {"account":account_name,
               "vmtype": 1,
               "vmversion":0,
               "code":code.hex()
               }
    eosapi.push_action('eosio', 'setcode', setcode, {account_name:'active'})
    db.remove_code(account_name)
    
    return True

def set_abi(account, abi):
    db.set_abi(account, abi)
    abi = _eosapi.pack_abi(abi)
    setabi ={'account':account, 'abi':abi.hex()}
    eosapi.push_action('eosio', 'setabi', setabi, {account:'active'})

print('done!')

done!


```python
def pack_uint32(n: int) -> bytes
def unpack_uint32(data: bytes) -> Tuple[int, int]
```

# string

In [86]:

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"worlds", "type":"string"},
      ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''
code = '''
import struct
def apply(receiver, code, action):
    data = read_action_data()
    print(data)
    n1, n2 = struct.unpack_uint32(data)
    print(n1, n2)
    print(data[n2:])
'''
account_name = 'helloworld11'

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'worlds':'hello,world'}
    r = eosapi.push_action(account_name, 'sayhello', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)


b'\x0bhello,world'
11 1
b'hello,world'

328


# bytes

In [87]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"worlds", "type":"bytes"},
      ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''
code = '''
import struct
def apply(receiver, code, action):
    data = read_action_data()
    n1, n2 = struct.unpack_uint32(data)
    print(n1, n2)
    print(data[n2:])
'''

account_name = 'helloworld11'
args = {'worlds':'a'*128}

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'worlds':bytes.hex(b'hello,world')}
    r = eosapi.push_action(account_name, 'sayhello', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)


11 1
b'hello,world'

345


# array

In [89]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"worlds", "type":"string[]"},
      ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = '''
import struct
def apply(receiver, code, action):
    data = read_action_data()
    print(data)
    n1, n2 = struct.unpack_uint32(data)
    index = n2
    for i in range(n1):
        n1, n2 = struct.unpack_uint32(data[index:])
        print(data[index+n2:index+n2+n1])
        index += n1
        index += n2
'''

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'worlds':['hello,world', 'hello']}
    r = eosapi.push_action(account_name, 'sayhello', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)


b'\x02\x0bhello,world\x05hello'
b'hello,world'
b'hello'

1244


# symbol

In [90]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "test",
      "base": "",
      "fields": [
        {"name":"test", "type":"symbol"},
      ]
    }
  ],
  "actions": [{
      "name": "test",
      "type": "test",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = '''
def apply(receiver, code, action):
    data = read_action_data()
    print(data)
'''

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'test':'4,SSSSSSS'}
    r = eosapi.push_action(account_name, 'test', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)



b'\x04SSSSSSS'

254


# asset
```c++
struct asset {
  int64_t      amount;
  symbol_type  symbol;
}
```

In [92]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "test",
      "base": "",
      "fields": [
        {"name":"test", "type":"asset"},
      ]
    }
  ],
  "actions": [{
      "name": "test",
      "type": "test",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = '''
import struct
def apply(receiver, code, action):
    data = read_action_data()
    amount, symbol = struct.unpack('q8s', data)
    print(amount, symbol)
'''

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'test':'100.0000 SYS'}
    r = eosapi.push_action(account_name, 'test', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)


1000000 b'\x04SYS\x00\x00\x00\x00'

1159


In [93]:
abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [{
      "new_type_name": "account_name",
      "type": "name"
   }],
  "structs": [{
      "name": "transfer",
      "base": "",
      "fields": [
        {"name":"from", "type":"account_name"},
        {"name":"to", "type":"account_name"},
        {"name":"quantity", "type":"asset"},
        {"name":"memo", "type":"string"}
      ]
    }
  ],
  "actions": [{
      "name": "transfer",
      "type": "transfer",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ],
  "ricardian_clauses": [],
  "abi_extensions": []
}
'''


code = '''
import struct
def apply(receiver, code, action):
    data = read_action_data()
    from_, to, amount, symbol = struct.unpack('QQq8s', data[:32])
    memo = data[32:]
    n1, n2 = struct.unpack_uint32(memo)
    memo = memo[n2:]
    print(memo)
    print(amount, symbol)
'''

set_abi(account_name, abi)
set_code(account_name, code)

try:
    args = {'from':'eosio', 'to':'eosio.token', 'quantity':'100.0000 UUOS', 'memo':'hello,world'}
    r = eosapi.push_action(account_name, 'transfer', args, {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print('error:', e)


b'hello,world'
1000000 b'\x04UUOS\x00\x00\x00'

1152
